<a href="https://colab.research.google.com/github/kiing-dom/CS4287/blob/main/Assignment1/CS4287_Prj1_20243561_20231717.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS4287 MLP

In [ ]:
# Dominion Gbadamosi - 20243561
# Ameen Popoola - 20231717

In [ ]:
#Imports
import numpy as np
import seaborn as sbn
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Visualization

In [ ]:
#Using Panda library to read the dataset
dataset = "Healthcare-Diabetes.csv"
df = pd.read_csv(dataset)

In [ ]:
#Display the first few rows of the dataset
display(df.head())

In [ ]:
#Display the dataset's statistics
display(df.describe())